In [14]:
import pytest
#Boîte A : Tous les titres contiennent le mot "notification" (différents émetteurs/destinataires)
#Boîte B : Tous les emails venant de l'émetteur blabla@gmail.com (différents titres/destinataires)

In [35]:
class Email_classifier():
    
    def classify(self, email):
        return 'boite B'

    def learn(self, dict):
        return ""

def test_classify():
    email_class = Email_classifier()
    boiteA = []
    boiteB = []
    faux_objet_email = {}

    faux_objet_email['emetteur'] = 'notif@sncf.com'
    faux_objet_email['destinataire'] = 'd@gmail.com'
    faux_objet_email['objet'] = 'notification'
    boiteA.append(faux_objet_email)

    faux_objet_email1 = {}
    faux_objet_email1['emetteur'] = 'test@gmail.com'
    faux_objet_email1['destinataire'] = 'm@gmail.com'
    faux_objet_email1['objet'] = 'notification'
    boiteA.append(faux_objet_email1)

    faux_objet_email2 = {}
    faux_objet_email2['emetteur'] = 'notification@gmail.com'
    faux_objet_email2['destinataire'] = 'mar@gmail.com'
    faux_objet_email2['objet'] = 'notification'
    boiteA.append(faux_objet_email2)

    faux_objet_email3 = {}
    faux_objet_email3['emetteur'] = 'test@gmail.com'
    faux_objet_email3['destinataire'] = 'blabla@gmail.com'
    faux_objet_email3['objet'] = 'blablabla chaussure'
    boiteB.append(faux_objet_email3)

    faux_objet_email4 = {}
    faux_objet_email4['emetteur'] = 'testonslesemails@gmail.com'
    faux_objet_email4['destinataire'] = 'blabla@gmail.com'
    faux_objet_email4['objet'] = 'blablabla tong'
    boiteB.append(faux_objet_email4)

    faux_objet_email5 = {}
    faux_objet_email5['emetteur'] = 'david@gmail.com'
    faux_objet_email5['destinataire'] = 'blabla@gmail.com'
    faux_objet_email5['objet'] = 'telephone'
    boiteB.append(faux_objet_email5)

    email_class.learn({'boite A': boiteA, 'boiteB': boiteB})
    assert email_class.classify(faux_objet_email) == 'boite A'



In [36]:
test_classify()

AssertionError: 